<table>
<tr>
<td width=15%><img src="assets/logo-uga.png"></img></td>
<td><center><h1> DashBike  <br>Data Visualisation</h1></center></td>
<td width=15%><a href="https://github.com/yanismicha/dashboard" style="font-size: 16px; font-weight: bold">Micha Yanis<br> Andrew Hardwick</a> </td>
</tr>
</table>

# Introduction
Ce projet de dashboard sur l'accidentologie de vélo en france est constitué de 4 principaux fichiers ainsi que d'un dossier assets pour toute les images et le style css utilisé. Les quatres fichiers sont:
- **dash.ipynb**: celui sur lequel vous vous situez qui regroupe le code permettant d'initialiser notre dashboard avec le squelette de celui-ci.
- **pages.py**: fichier contenant l'ensemble des pages avec tout les éléments *ui* constituant le dashboard.
- **Figure.py**: contient l'ensemble des figures utilisés, les graphiques, les cartes, ainsi que les transformations nécessaire de nos données pour permettre une meilleure visualisation (changement de variable,palette de couleurs,...). Le fichier contient également des fonctions utilisés pour ces transformations.
- **Functions.py**: contient l'ensemble de la partie serveur de notre dashboard.

Le Dashboard se base sur un ensemble de jeu de données concernant l'accidentologie de vélo en France:
- **accidents-velos_clean.csv**:le principal jeu de données utilisé, correspond aux accidents de vélos en france entre 2005 et 2021.
- **pop_par_dep.csv**,**pop-regions.csv**, contient des informations concernant les départements et régions françaises.
- **pistes_com.csv**,**pistes_dep.csv**,**pistes_reg.csv**, contient les informations sur la quantité de pistes cyclables dans les zones en question.
  
<div class="alert alert-warning">
    <span class="badge badge-warning"IMPORTANTg</span>
  Il arrive qu'un graphique de la première page ne se lance pas, nous n'avons pas encore réussi à le débuguer, néanmoins, actualiser le dashboard suffit à le faire fonctionner corretement!t.
</drts

## Les imports

In [1]:
import dash
import dash_bootstrap_components as dbc
from dash import Dash,dcc,html,callback,Input,Output, Patch, clientside_callback, callback
import plotly.io as pio
from dash_bootstrap_templates import load_figure_template
import pandas as pd 
import plotly.express as px
import Functions_dash as Fun
import Figure as fig
import pages as pag
from PIL import Image
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Notre data

In [3]:
data= pd.read_csv("accidents-velos_clean.csv",low_memory=False)

# Notre application

In [2]:
app = dash.Dash(
    external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME, "assets/styles.css"],suppress_callback_exceptions=True
)

# Bar noir en haut de page
navbar = dbc.NavbarSimple(
    children=[
        dbc.Button("Menu", outline=True, color="secondary", className="mr-1", id="btn_sidebar"), # Bouton pour le menu
        dbc.Popover(
                    "CLiquez pour ranger/sortir le sidebar",
                    target='btn_sidebar',
                    trigger="hover",
                    placement = "bottom"
                 ), 
        # Select for merge
        dcc.Dropdown(
                    id='zone-data-filter',
                    options=[{"label":"Nationale","value":'all'},
                             {"label":"Régional","value":'reg'},
                             {"label":"Département","value":'dep'}
                            ],
                    value="all",
                    searchable=False,
                    clearable=False,
                    ),
        dbc.Popover([dbc.PopoverHeader("Choix de zone géographique"),
                     dbc.PopoverBody("Vous pouvez choisir la zone géographique à visualiser pour tout le dashboard")
                     ],
                    target='zone-data-filter',
                    trigger="hover",
                    placement = "right",
                    id="popover-zone-geo",
                    ),
        dcc.Dropdown(
                    id='zone-selection',
                    options=[],
                    value=None,
                    searchable=False,
                    clearable=False,
                    placeholder=""
                    )
    ],
    brand="",
    brand_href="#",
    color="dark",
    dark=True,
    fluid=True,
    links_left= True,
)


submenu_1 = [
    html.Li(
        # use Row and Col components to position the chevrons
        dbc.Row(
            [
                dbc.Col("Visualisations"),
                dbc.Col(
                    html.I(className="fas fa-chevron-right me-3"),
                    width="auto",
                ),
            ],
            className="my-1",
        ),
        style={"cursor": "pointer"},
        id="submenu-1",
    ),
    # we use the Collapse component to hide and reveal the navigation links
    dbc.Collapse(
        [
            dbc.NavLink("Evolution temporelle", href="/page-1/1"),
            dbc.NavLink("Caractéristiques des accidents", href="/page-1/2"),
        ],
        id="submenu-1-collapse",
    ),
]

submenu_2 = [
    html.Li(
        dbc.Row(
            [
                dbc.Col("Cartes interactives"),
                dbc.Col(
                    html.I(className="fas fa-chevron-right me-3"),
                    width="auto",
                ),
            ],
            className="my-1",
        ),
        style={"cursor": "pointer"},
        id="submenu-2",
    ),
    dbc.Collapse(
        [
            dbc.NavLink("Carte de la France", href="/page-map"),
            dbc.NavLink("Carte par région/départemnt", href="/page-2/2"),
        ],
        id="submenu-2-collapse",
    ),
]


sidebar = html.Div(
    [
        html.H2("DashBike", className="display-4"),
        html.Hr(),
        html.Img(src = Image.open("assets/accident_bike.png"),style={"width": "60%",
                                                             "margin-bottom": '10%'}),
        html.P(
            "Vous trouverez ici les différentes pages du dashboard", className="lead"
        ),
        dbc.Nav(submenu_1 + submenu_2, vertical=True,pills=True),
        html.Img(src=Image.open("assets/roue.png"),style={"width": "60%",
                                                  "margin-top": '140%'}),
    ],
    #style=fig.SIDEBAR_STYLE,
    className="SIDEBAR_STYLE",
    id="sidebar",
)


content = html.Div(id="page-content", 
                   #style=fig.CONTENT_STYLE,
                   className="CONTENT_STYLE")

app.layout = html.Div(
    [
        dcc.Store(id='side_click'),
        dcc.Store(id='data-store', storage_type='session'),
        dcc.Location(id="url"),
        navbar,
        sidebar,
        content,
    ],
)



# Fonction qui appel toute les fonctions callback qui sont dans le fichier Fonction_dash.py
Fun.get_callbacks(app)

if __name__ == "__main__":
    app.run_server(debug=True,jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File ~\Documents\yanis\cours\Master ssd\M1\Semestre_2\Visualisations\Figure.py:355, in fig2(
    speed_animation='normal',
    data=       Unnamed: 0.1  Unnamed: 0       Num_Acc   ...
93085  Mayotte   06  

[93086 rows x 47 columns]
)
    353 def fig2(speed_animation, data: pd.DataFrame = data):
    354     accidents_par_annee_mois = data.groupby(['an','mois'], observed=False).size().reset_index(name = 'Nombre_d_accidents')
--> 355     fig2 = px.line(accidents_par_annee_mois, x="mois", y="Nombre_d_accidents", 
        accidents_par_annee_mois =        an       mois  Nombre_d_accidents
0    2005    janvier                 363
1    2005    février                 242
2    2005       mars                 381
3    2005      avril                 435
4    2005        mai                 685
..    ..